In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import json
import gzip

def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)

In [2]:
df_gpa = pd.read_csv("https://github.com/wadefagen/datasets/raw/master/gpa/uiuc-gpa-dataset.csv").rename(columns={"Year":"year","Term":"term","Subject":"subject","Number":"number","Primary Instructor":"instructor"})

df_gpa["instructor"] = df_gpa['instructor'].str.extract(r'(\w+, \w)')

df_gpa['total_students'] = df_gpa['A+'] + df_gpa['A'] + df_gpa['A-'] + df_gpa['B'] + df_gpa['B+'] + df_gpa['B-'] + df_gpa['C+'] + df_gpa['C'] + df_gpa['C-'] + df_gpa['D+'] + df_gpa['D'] + df_gpa['D-'] + df_gpa['F']

df_gpa['gpa'] = (df_gpa['A+'] * 4 + df_gpa['A'] * 4 + df_gpa['A-'] * 3.67 + df_gpa['B'] * 3 + df_gpa['B+'] * 3.33 + df_gpa['B-'] * 2.67 + df_gpa['C+'] * 2.33 + df_gpa['C'] * 2 + df_gpa['C-'] * 1.67 + df_gpa['D+'] * 1.33 + df_gpa['D'] + df_gpa['D-'] * 0.67) / df_gpa['total_students']

df_gpa = df_gpa.groupby(["year", "term", "subject", "number", "instructor"], as_index=False).agg({"gpa": "mean", "total_students": "sum", "A+": "sum", "A": "sum", "A-": "sum", "B+": "sum", "B": "sum", "B-": "sum", "C+": "sum", "C": "sum", "C-": "sum", "D+": "sum", "D": "sum", "D-": "sum", "F": "sum"})

df_gpa["course"] = df_gpa["subject"] + " " + df_gpa["number"].astype(str)

df_gpa

,year,term,subject,number,instructor,gpa,total_students,A+,A,A-,...,B,B-,C+,C,C-,D+,D,D-,F,course
0,2010,Fall,AAS,100,"Arnaldo, C",3.463613,69,0,22,21,...,8,3,1,2,1,0,0,1,0,AAS 100
1,2010,Fall,AAS,100,"Kwon, Y",3.358982,61,6,10,14,...,5,2,3,1,0,0,0,1,1,AAS 100
2,2010,Fall,AAS,100,"Manalansan, M",3.980294,34,21,12,0,...,0,0,0,0,0,0,0,0,0,AAS 100
3,2010,Fall,AAS,100,"Winkelmann, M",3.422059,34,1,12,11,...,0,0,1,2,0,0,1,0,1,AAS 100
4,2010,Fall,AAS,120,"Lee, A",3.127315,65,8,11,3,...,13,5,2,5,1,0,4,0,0,AAS 120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39791,2020,Summer,TAM,212,"Chang, W",2.808077,52,5,7,7,...,6,6,0,5,2,3,1,0,4,TAM 212
39792,2020,Summer,TAM,251,"Kim, S",3.417083,48,5,14,8,...,10,2,1,3,1,0,0,0,0,TAM 251
39793,2020,Summer,TAM,335,"Ramlawi, N",3.097500,24,2,4,4,...,3,3,1,1,0,0,1,0,1,TAM 335
39794,2020,Summer,THEA,101,"Morrissette, J",3.586707,59,35,4,4,...,2,5,2,1,0,1,0,0,1,THEA 101


In [51]:
terms = [
    "../raw/{}-{}.csv".format(year, term)
    for year in range(df_gpa["year"].min(), df_gpa["year"].max() + 2) 
      for term in ["Winter", "Spring", "Summer", "Fall"] 
        if Path("../raw/{}-{}.csv".format(year, term)).is_file()
]

df_catalogs = pd.concat([pd.read_csv(term) for term in terms], ignore_index=True)
df_catalogs["term"] = pd.Categorical(df_catalogs["term"], ["Fall","Summer","Spring","Winter"], ordered=True)
df_catalogs.sort_values(by=["year", "term", "subject", "number", "crn", "meeting"], ascending=[False, True, True, True, True, True], ignore_index=True, inplace=True)
df_catalogs["course"] = df_catalogs["subject"] + " " + df_catalogs["number"].astype(str)

# Fix typos in descriptions
df_catalogs.loc[df_catalogs["course"]=="HIST 574", "description"]="Immerses students in major works of recent American religious history. Written from multiple disciplinary perspectives and wrestling with the knotty problems in which religion has been interwoven, these books will give the student a solid foundation in American religious history. 4 graduate hours. No professional credit."
df_catalogs.loc[df_catalogs["course"]=="ASST 104", "description"] = "Same as REL 104. See REL 104."
df_catalogs.loc[df_catalogs["course"]=="EPOL 551", "description"] = "Same as EOL 570. See EOL 570."

df_catalogs = df_catalogs.merge(df_gpa, how="left", on=["year", "term", "course", "subject", "number", "instructor"])

df_catalogs = trim_all_columns(df_catalogs)

df_catalogs

,year,term,college,subject,subject_name,number,name,description,credit_hours,gen_ed,...,B+,B,B-,C+,C,C-,D+,D,D-,F
0,2021,Spring,KV,AAS,Asian American Studies,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.,1US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021,Spring,KV,AAS,Asian American Studies,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.,1SS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,Spring,KV,AAS,Asian American Studies,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.,1US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021,Spring,KV,AAS,Asian American Studies,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.,1SS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021,Spring,KV,AAS,Asian American Studies,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.,1US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243508,2010,Spring,KU,LAW,Law,636,Business Associations II,This is the second course in the sequence (off...,3 OR 4 hours.,NaN,...,29.0,8.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
243509,2010,Spring,KU,LAW,Law,639,Corporate Finance,Analysis of corporate and securities law probl...,3 OR 4 hours.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243510,2010,Spring,KU,LAW,Law,639,Corporate Finance,Analysis of corporate and securities law probl...,3 OR 4 hours.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243511,2010,Spring,KU,LAW,Law,647,Income Taxation,The fundamental course in federal income taxat...,3 OR 4 hours.,NaN,...,12.0,10.0,5.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0


In [38]:
df_colleges = (pd.read_json("../Colleges.json", orient="index")
               .reset_index()
               .rename(columns={"index": "collegeId", 0: "name"}))
df_colleges.to_csv("../neo4j/nodes/college_nodes.csv", index=False)
df_colleges

,collegeId,name
0,KL,"Agricultural, Consumer and Environmental Sciences"
1,KY,Applied Health Sciences
2,LD,Armed Forces
3,KT,College of Media
4,KW,Division of General Studies
5,KN,Education
6,KP,Engineering
7,LK,Environmental Council
8,KR,Fine and Applied Arts
9,KM,Gies College of Business


In [39]:
df_subjects = (df_catalogs[["subject", "subject_name"]]
               .drop_duplicates(ignore_index=True)
               .dropna()
               .rename(columns={"subject": "subjectId", "subject_name": "name"}))
df_subjects.to_csv("../neo4j/nodes/subject_nodes.csv", index=False)
df_subjects

,subjectId,name
0,AAS,Asian American Studies
1,ABE,Agricultural and Biological Engineering
2,ACCY,Accountancy
3,ACE,Agricultural and Consumer Economics
4,ACES,"Agricultural, Consumer and Environmental Sciences"
...,...,...
238,GWS,Gender and Women's Studies Program
239,HEBR,"Religion, Program for the Study of"
240,LGLA,Linguistics
241,VB,Veterinary Biosciences


In [40]:
df_courses = (df_catalogs[["course", "number", "name", "description", "credit_hours"]]
              .drop_duplicates(["course"], ignore_index=True)
              .dropna()
              .set_index(["course"]))
df_bad_descriptions = df_courses.loc[df_courses["description"].str.extract(r"See\s*([A-Z]{2,4}\s*[0-9]{3})").dropna().index]
df_bad_descriptions["see_course"] = df_bad_descriptions["description"].str.extract(r"See\s*([A-Z]{2,4}\s*[0-9]{3})")[0].values
df_bad_descriptions["better_description"] = df_courses.loc[df_bad_descriptions["see_course"].values, "description"].values
df_courses.loc[df_bad_descriptions.index, "description"] = df_bad_descriptions.apply(lambda row: row["better_description"].replace(row.name, row["see_course"]), axis=1)
df_courses.reset_index(inplace=True)
df_courses.to_csv("../neo4j/nodes/course_nodes.csv", index=False)
df_courses

,course,number,name,description,credit_hours
0,AAS 100,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.
1,AAS 201,201,US Racial & Ethnic Politics,Examines efforts by racial and ethnic communit...,3 hours.
2,AAS 215,215,US Citizenship Comparatively,"Examines the racial, gendered, and sexualized ...",3 hours.
3,AAS 246,246,Asian American Youth in Film,Examines both mainstream and independent films...,3 hours.
4,AAS 258,258,Muslims in America,Introduction to the study of Muslims in the Un...,3 hours.
...,...,...,...,...,...
10189,VCM 658,658,Clinical Procedure/Problem I,Course is designed to train students in physic...,1 hours.
10190,VCM 683,683,Advanced Soft Tissue Surgery,"Advanced instruction in the pathophysiology, d...",1 hours.
10191,VCM 684,684,Client Relations,"Introduction to client relations, including te...",1 hours.
10192,VCM 691,691,Adv Orthopedics Fract Fixation,Advanced instruction in the pathophysiology of...,1 hours.


In [60]:
df_sections = (df_catalogs[["crn", "year", "term", "part_of_term", 
                           "gpa", "A+", "A", "A-", "B+", "B", "B-",
                           "C+", "C", "C-", "D+", "D", "D-", "F",
                           "section_info", "section_notes", 
                           "section_attributes", "section_capp_area", 
                           "section_co_request", "section_special_approval"]]
              .drop_duplicates(ignore_index=True)
              .dropna(subset=["crn", "year", "term"]))
df_sections[["A+", "A", "A-", "B+", "B", "B-","C+", "C", "C-", "D+", "D", "D-", "F"]] = df_sections[["A+", "A", "A-", "B+", "B", "B-","C+", "C", "C-", "D+", "D", "D-", "F"]].fillna(0)
df_sections.replace([np.nan], [None], inplace=True)
df_sections.to_csv("../neo4j/nodes/section_nodes.csv", index=False)
df_sections

,crn,year,term,part_of_term,gpa,A+,A,A-,B+,B,...,D+,D,D-,F,section_info,section_notes,section_attributes,section_capp_area,section_co_request,section_special_approval
0,30106,2021,Spring,1,None,0,0,0,0,0,...,0,0,0,0,None,None,"Social & Beh Sci - Soc Sci, and Cultural Studi...",None,None,None
1,30107,2021,Spring,1,None,0,0,0,0,0,...,0,0,0,0,None,None,"Social & Beh Sci - Soc Sci, and Cultural Studi...",None,None,None
2,41729,2021,Spring,1,None,0,0,0,0,0,...,0,0,0,0,None,None,"Social & Beh Sci - Soc Sci, and Cultural Studi...",None,None,None
3,43832,2021,Spring,1,None,0,0,0,0,0,...,0,0,0,0,None,None,"Social & Beh Sci - Soc Sci, and Cultural Studi...",None,None,None
4,48232,2021,Spring,1,None,0,0,0,0,0,...,0,0,0,0,None,None,"Social & Beh Sci - Soc Sci, and Cultural Studi...",None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179905,44950,2010,Spring,LF,3.4595,2,7,20,29,8,...,0,0,0,0,None,None,None,None,None,None
179906,47032,2010,Spring,LF,None,0,0,0,0,0,...,0,0,0,0,None,Not intended for Master of Laws.,None,None,None,None
179907,47525,2010,Spring,LF,None,0,0,0,0,0,...,0,0,0,0,None,None,None,None,None,None
179908,35927,2010,Spring,LF,3.29146,1,10,6,12,10,...,0,0,0,0,None,Not intended for Master of Laws.,None,None,None,None


In [61]:
df_meetings = df_catalogs[["start_date", "end_date", "start_time", "end_time", "type", "type_name", "days", "room", "building"]]

KeyError: "['typeId'] not in index"